# TG涡

函数 \(a(x, y)\)、\(b(x, y)\)、\(c(x, y)\) 和 \(d(x, y)\) 定义如下：
\begin{align*}
    H(x) &= 
    \begin{cases} 
      \frac{x}{2} & \text{如果 } 0 \leq x \leq \pi \\
      \pi - \frac{x}{2} & \text{如果 } \pi < x \leq 2\pi 
    \end{cases} \\
    a(x, y) &= \cos[H(x)] \cos\left(\frac{\cos(y) (2 - \cos(x))}{\hbar}\right), \\
    b(x, y) &= \cos[H(x)] \sin\left(\frac{\cos(y) (2 - \cos(x))}{\hbar}\right), \\
    c(x, y) &= \sin[H(x)] \cos\left(\frac{\cos(y) (2 + \cos(x))}{\hbar}\right), \\
    d(x, y) &= -\sin[H(x)] \sin\left(\frac{\cos(y) (2 + \cos(x))}{\hbar}\right).
\end{align*}
其中，$\hbar $是一个常数（为了简化，设为1）。

速度场计算
$$
\mathbf{u} = \hbar(a \mathbf \nabla b-b\mathbf \nabla a + c\mathbf \nabla d - d\mathbf \nabla c)
$$

速度场分量 $u_x$ 和 $u_y$ 使用 $a$、$b$、$c$、$d$ 的梯度计算得到：
$$
\begin{align*}
    \nabla a &= \left(\frac{\partial a}{\partial x}, \frac{\partial a}{\partial y}\right), \\
    \nabla b &= \left(\frac{\partial b}{\partial x}, \frac{\partial b}{\partial y}\right), \\
    \nabla c &= \left(\frac{\partial c}{\partial x}, \frac{\partial c}{\partial y}\right), \\
    \nabla d &= \left(\frac{\partial d}{\partial x}, \frac{\partial d}{\partial y}\right).
\end{align*}
$$

速度场的计算公式为：
\begin{align*}
    u_x &= \hbar \left(a \frac{\partial b}{\partial x} - b \frac{\partial a}{\partial x} + c \frac{\partial d}{\partial x} - d \frac{\partial c}{\partial x}\right), \\
    u_y &= \hbar \left(a \frac{\partial b}{\partial y} - b \frac{\partial a}{\partial y} + c \frac{\partial d}{\partial y} - d \frac{\partial c}{\partial y}\right).
\end{align*}

In [5]:
import numpy as np
import matplotlib.pyplot as plt


In [6]:
# 定义常数
hbar = 1

# 定义 H(x)
def H(x):
    return np.where((0 <= x) & (x <= np.pi), x / 2, np.pi - x / 2)

# 定义函数 a, b, c, d
def a(x, y):
    return np.cos(H(x)) * np.cos(np.cos(y) * (2 - np.cos(x)) / hbar)

def b(x, y):
    return np.cos(H(x)) * np.sin(np.cos(y) * (2 - np.cos(x)) / hbar)

def c(x, y):
    return np.sin(H(x)) * np.cos(np.cos(y) * (2 + np.cos(x)) / hbar)

def d(x, y):
    return -np.sin(H(x)) * np.sin(np.cos(y) * (2 + np.cos(x)) / hbar)

# 计算梯度
def gradient(f, x, y, eps=1e-5):
    f_x = (f(x + eps, y) - f(x - eps, y)) / (2 * eps)
    f_y = (f(x, y + eps) - f(x, y - eps)) / (2 * eps)
    return f_x, f_y



In [7]:
# 计算速度场
def compute_velocity_field_np(x, y, dx, dy):
    # 使用内置的np 求解梯度
    a_val = a(x, y)
    b_val = b(x, y)
    c_val = c(x, y)
    d_val = d(x, y)

    a_grad_x, a_grad_y = np.gradient(a_val, dx, dy)
    b_grad_x, b_grad_y = np.gradient(b_val, dx, dy)
    c_grad_x, c_grad_y = np.gradient(c_val, dx, dy)
    d_grad_x, d_grad_y = np.gradient(d_val, dx, dy)

    u_x = hbar * (a_val * b_grad_x - b_val * a_grad_x + c_val * d_grad_x - d_val * c_grad_x)
    u_y = hbar * (a_val * b_grad_y - b_val * a_grad_y + c_val * d_grad_y - d_val * c_grad_y)

    return u_x, u_y

此处我们设定low像素:   $64\times 64$

设定High像素为    $1024\times 1024$

我们最终的目标是进行16倍的超分辨

In [8]:
size_low = 64
size_High = 1024
x_low = np.linspace(0, 2 * np.pi, size_low)
y_low = np.linspace(0, 2 * np.pi, size_low)

x_high = np.linspace(0, 2 * np.pi, size_High)
y_high = np.linspace(0, 2 * np.pi, size_High)


X_low, Y_low = np.meshgrid(x_low, y_low)
X_high, Y_high = np.meshgrid(x_high, y_high)

In [9]:
# 计算函数值
A_low = a(X_low, Y_low)
B_low = b(X_low, Y_low)
C_low = c(X_low, Y_low)
D_low = d(X_low, Y_low)

A_high = a(X_high, Y_high)
B_high = b(X_high, Y_high)
C_high = c(X_high, Y_high)
D_high = d(X_high, Y_high)


In [10]:
# 计算步长
dx_low = x_low[1] - x_low[0]
dy_low = y_low[1] - y_low[0]

dx_high = x_high[1] - x_high[0]
dy_high = y_high[1] - y_high[0]

In [16]:
print("查看ABCD低分辨率的形状")
print(A_low.shape,B_low.shape,C_low.shape,D_low.shape)

print("查看ABCD高分辨率的形状")
print(A_high.shape,B_high.shape,C_high.shape,D_high.shape)

print("dx的形状")
print(dx_low,dy_low)
print(dx_high,dy_high)


查看ABCD低分辨率的形状
(64, 64) (64, 64) (64, 64) (64, 64)
查看ABCD高分辨率的形状
(1024, 1024) (1024, 1024) (1024, 1024) (1024, 1024)
dx的形状
0.09973310011396169 0.09973310011396169
0.00614192112138767 0.00614192112138767
